In [3]:
def write_code(flag=True):
    if flag:
        code = """print("Flag is true")
        """
    else:
        code = """print("Flag is false")
        """
        
    return code

exec(write_code(False))

Flag is false
